# ++ Carregar dados históricos da Bovespa em Dataframe para análise ++

In [1]:
import time
time.asctime( time.localtime(time.time()) )

'Tue Apr 16 22:50:33 2024'

In [2]:
import pandas as pd
from datetime import date, datetime, timedelta
# pd.__version__

In [3]:
from IPython.display import FileLink, FileLinks
FileLink('Resultados.ipynb')

C:\Users\esmer\Desktop\Alexandre\Projeto Git\StockManager\Resultados.ipynb

In [4]:
# Setando opção do Pandas para mostrar todas as linhas
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Setando separador de milhar
pd.options.display.float_format = '{:,.2f}'.format

In [5]:
from pacote.functions import *

In [6]:
ano = time.localtime().tm_year
str_ano = f'A{ano}'

if time.localtime().tm_mon < 3:
    str_nov_ant, str_dez_ant = f'M11{ano-1}', f'M12{ano-1}'
    LISTA_ARQUIVOS = [str_ano, str_nov_ant, str_dez_ant]
else:
    LISTA_ARQUIVOS = [str_ano]

In [7]:
df = carrega_dados(LISTA_ARQUIVOS)

In [8]:
from IPython.display import Markdown, display

# Monday is 0 and Sunday is 6.
dia_semana  = datetime.today().weekday()
ajuste_dias = -3 if dia_semana == 0 else -1
dt_esperado_ult = (datetime.today() + timedelta(days=ajuste_dias)).strftime('%Y-%m-%d')
dt_max = str(df["dtPregao"].max()).split()[0]

if dt_esperado_ult == dt_max:
    font_color="alert-success"
    a = "Atualizado!"
else:
    font_color="alert-danger"
    a = "Desatualizado!!! Última data " + str(dt_max)
def printmd(string):
    display(Markdown(string))

In [9]:
#  df_n_dias | vol | col_pc | pc_min | avg_vr_fech
d25 = monta_tabela(monta_df_periodos(df, 25), 3000000, '1.0%', 22, 2)
d20 = monta_tabela(monta_df_periodos(df, 20), 3000000, '1.0%', 18, 2)
d15 = monta_tabela(monta_df_periodos(df, 15), 3000000, '1.0%', 14, 2)
d10 = monta_tabela(monta_df_periodos(df, 10), 3000000, '1.0%', 9, 2)
d05 = monta_tabela(monta_df_periodos(df, 5), 3000000, '1.0%', 5, 1)
d04 = monta_tabela(monta_df_periodos(df, 4), 3000000, '1.0%', 4, 1)
d03 = monta_tabela(monta_df_periodos(df, 3), 3000000, '1.0%', 3, 1)
merge1 = pd.merge(d25['cdAcao'], d20['cdAcao'], how ='inner', on =['cdAcao'])
merge2 = pd.merge(merge1, d15['cdAcao'], how ='inner', on =['cdAcao'])
merge3 = pd.merge(merge2, d10['cdAcao'], how ='inner', on =['cdAcao'])
merge_final = pd.merge(merge3, d05['cdAcao'], how ='inner', on =['cdAcao'])

In [85]:
def filtra_data(df, data="max"):
    # formato da data: 'aaaa-mm-dd'
    dt_max = df["dtPregao"].max() if data == "max" else datetime.strptime(data, '%Y-%m-%d')
    df_max_dt = df.loc[df["dtPregao"] == dt_max]
    return df_max_dt

In [90]:
data = "2024-04-04"
df_max_dt = filtra_data(df, data)

df_2max_dt = df.copy()
df_2max_dt = df_2max_dt.loc[df_2max_dt["dtPregao"] < df_max_dt.iloc[0, 1]]
df_2max_dt = filtra_data(df_2max_dt)

df_max_dt = df_max_dt[["cdAcao", "vrVolume", "pcVar", "dtPregao"]]
df_2max_dt = df_2max_dt[["cdAcao", "vrVolume", "pcVar", "dtPregao"]]

merge_max = pd.merge(df_2max_dt[['cdAcao','dtPregao','vrVolume','pcVar']], df_max_dt[['cdAcao','vrVolume','pcVar','dtPregao']], how ='inner', on =['cdAcao'])
merge_max = merge_max.loc[(merge_max["vrVolume_x"] > 1000000) & 
                          (merge_max["vrVolume_y"] > merge_max["vrVolume_x"] * 3)]

In [10]:
printmd(f'**<div class={font_color}>Arquivo {a}</div>**')

**<div class=alert-danger>Arquivo Desatualizado!!! Última data 2024-04-16</div>**

### 25 dias, order by 1%

In [11]:
d25

,cdAcao,0.5%,1.0%,1.5%,2.0%,2.5%,3.0%,AvgVol,AvgVrFech,AvgPcAbert
264,CLSA3,24,23,22,21,18,16,"8,549,449.76",5.22,0.14


### 20 dias, order by 1%

In [12]:
d20

,cdAcao,0.5%,1.0%,1.5%,2.0%,2.5%,3.0%,AvgVol,AvgVrFech,AvgPcAbert
257,CLSA3,19,18,18,17,15,13,"10,013,025.25",5.54,0.11


### 15 dias, order by 1%

In [13]:
d15

,cdAcao,0.5%,1.0%,1.5%,2.0%,2.5%,3.0%,AvgVol,AvgVrFech,AvgPcAbert


### 10 dias, order by 1%

In [14]:
d10

,cdAcao,0.5%,1.0%,1.5%,2.0%,2.5%,3.0%,AvgVol,AvgVrFech,AvgPcAbert
212,CBAV3,9,9,8,8,7,7,"28,128,936.80",4.69,0.39
717,PETR4,9,9,5,3,3,0,"2,057,426,156.40",38.84,0.12


### 5 dias, order by 1%

In [15]:
d05

,cdAcao,0.5%,1.0%,1.5%,2.0%,2.5%,3.0%,AvgVol,AvgVrFech,AvgPcAbert
81,AZEV4,5,5,5,5,5,5,"14,717,639.40",1.66,-0.53
395,GOGL34,5,5,4,2,1,0,"15,254,186.80",67.23,0.39
601,NFLX34,5,5,3,3,1,1,"4,220,798.04",63.74,0.38


In [16]:
d04

,cdAcao,0.5%,1.0%,1.5%,2.0%,2.5%,3.0%,AvgVol,AvgVrFech,AvgPcAbert
81,AZEV4,4,4,4,4,4,4,"14,134,514.25",1.65,-0.67
188,CBAV3,4,4,4,4,3,3,"24,842,830.75",4.83,0.58
35,AAPL34,4,4,4,2,2,1,"12,584,077.56",44.72,0.40
535,MELI34,4,4,3,3,1,1,"56,616,147.64",62.19,0.14
584,NFLX34,4,4,3,3,1,1,"4,717,585.69",63.97,0.53
386,GOGL34,4,4,3,2,1,0,"11,566,690.39",67.51,0.49


In [17]:
d03

,cdAcao,0.5%,1.0%,1.5%,2.0%,2.5%,3.0%,AvgVol,AvgVrFech,AvgPcAbert
77,AZEV4,3,3,3,3,3,3,"12,243,929.00",1.61,-0.10
182,CBAV3,3,3,3,3,2,2,"28,574,531.33",4.82,0.70
32,AAPL34,3,3,3,1,1,0,"12,331,228.93",44.81,0.37
519,MELI34,3,3,2,2,1,1,"65,503,571.89",61.54,0.61
567,NFLX34,3,3,2,2,1,1,"4,203,452.33",63.90,0.83
373,GOGL34,3,3,2,1,0,0,"12,741,472.62",67.40,0.53
620,PETR3,3,3,2,0,0,0,"471,103,958.00",40.76,0.02
364,GFSA3,3,3,1,1,1,1,"14,939,892.67",5.97,-0.12
621,PETR4,3,3,1,0,0,0,"1,383,585,445.33",39.25,0.10


In [18]:
printmd(f'**<div class={font_color}>Status {a}</div>**')

**<div class=alert-danger>Status Desatualizado!!! Última data 2024-04-16</div>**

### Consulta ação específica

In [93]:
consulta_acao(df, "PRNR3")

,cdAcao,dtPregao,vrFech,vrVolume,vrMaxDia,vrMinDia,vrAbert,pcVar,pcMaxDia,pcMinDia,pcAbert,ic05,ic10,ic15,ic20,ic25,ic30
54325,PRNR3,2024-04-16,12.40,"4,695,257",12.65,12.22,12.57,-1.82,0.16,-3.25,-0.48,,,,,,
53423,PRNR3,2024-04-15,12.63,"7,750,977",13.48,12.60,13.37,-5.46,0.90,-5.69,0.07,1,,,,,
52514,PRNR3,2024-04-12,13.36,"12,475,413",13.48,13.11,13.46,-1.47,-0.59,-3.32,-0.74,,,,,,
51906,PRNR3,2024-04-11,13.56,"4,914,255",13.64,13.31,13.59,-0.73,-0.15,-2.56,-0.51,,,,,,
51299,PRNR3,2024-04-10,13.66,"4,672,645",13.69,13.42,13.60,0.96,1.18,-0.81,0.52,1,1,,,,
50479,PRNR3,2024-04-09,13.53,"12,242,748",13.59,13.04,13.28,3.44,3.90,-0.31,1.53,1,1,1,1,1,1
49830,PRNR3,2024-04-08,13.08,"12,553,557",13.18,12.40,12.50,1.32,2.09,-3.95,-3.18,1,1,1,1,,
49016,PRNR3,2024-04-05,12.91,"11,148,739",13.00,12.40,12.44,3.28,4.00,-0.80,-0.48,1,1,1,1,1,1
48139,PRNR3,2024-04-04,12.50,"10,597,700",12.57,11.88,11.91,4.87,5.45,-0.34,-0.08,1,1,1,1,1,1
47461,PRNR3,2024-04-03,11.92,"3,372,495",12.07,11.63,11.83,1.45,2.72,-1.02,0.68,1,1,1,1,1,


### Ações presentes em todos os períodos

In [20]:
merge_final

,cdAcao


In [94]:
merge_max

,cdAcao,dtPregao_x,vrVolume_x,pcVar_x,vrVolume_y,pcVar_y,dtPregao_y
56,AZEV4,2024-04-03,"1,649,562.00",1.89,"16,983,086.00",23.15,2024-04-04
134,CBAV3,2024-04-03,"10,752,520.00",0.25,"33,909,954.00",7.16,2024-04-04
360,LIGT3,2024-04-03,"3,658,987.00",0.77,"20,118,638.00",9.94,2024-04-04
440,OIBR3,2024-04-03,"2,580,087.00",-1.54,"15,275,113.00",4.69,2024-04-04
463,PETR3,2024-04-03,"454,662,328.00",-0.78,"1,405,401,820.00",-0.46,2024-04-04
464,PETR4,2024-04-03,"1,210,506,567.00",-0.52,"6,399,041,382.00",-1.41,2024-04-04
482,PRNR3,2024-04-03,"3,372,495.00",1.45,"10,597,700.00",4.87,2024-04-04
538,SANB3,2024-04-03,"1,018,698.00",-0.30,"4,829,449.00",1.36,2024-04-04
539,SANB4,2024-04-03,"1,278,518.00",-1.01,"5,419,839.00",2.91,2024-04-04


### ======================================== TESTES ========================================

In [21]:
bla

NameError: name 'bla' is not defined

### Comparar Maiores Lucros

In [ ]:
df_original = df.copy()

In [ ]:
df = df_original.copy()

In [ ]:
df_teste = df_original.copy()

In [ ]:
dt_max = df_teste["dtPregao"].max()
df_teste = df_teste.loc[df_teste["dtPregao"] != dt_max]

In [ ]:
d05 = monta_tabela(monta_df_periodos(df_teste, 5), 3000000, '1.0%', 5, 2)
d05

In [ ]:
consulta_acao(df_teste, "RAIZ4")

### ----------------------------------------------------------------------------

In [ ]:
df_1 = monta_lucro_periodo(df, 10, 0, True)
df_2 = monta_lucro_periodo(df, 10, 1, True)

In [ ]:
df_final = df_1.copy()
df_final['posicao ant'], df_final['mudanca'] = [
    df_2['posicao']
    , df_2['posicao'] - df_1['posicao']
]
df_final['posicao ant'] = df_final['posicao ant'].fillna(0).astype('int')
df_final['mudanca'] = df_final['mudanca'].fillna(0).astype('int')

In [ ]:
df_final.sort_values(["mudanca"], ascending=False)

In [ ]:
df_final.sort_values(["mudanca"], ascending=False).loc[df_final['posicao ant'] <= 10]

In [ ]:
df_1

In [ ]:
df_2

In [ ]:
# dfDtMin = df5d.loc[(df["dtPregao"] == dt_min_5d) & (df["cdAcao"] == 'PTBL3')]

df5d  = busca_periodos(df, 5)
df25d = busca_periodos(df, 25)

dt_min_5d  = df5d["dtPregao"].min()
dt_min_25d = df25d["dtPregao"].min()
dt_max     = df5d["dtPregao"].max()
dfDtMin5d  = df5d.loc[(df5d["dtPregao"] == dt_min_5d)].set_index(["cdAcao"])
dfDtMin25d = df25d.loc[(df25d["dtPregao"] == dt_min_25d)].set_index(["cdAcao"])
dfDtMax    = df5d.loc[(df5d["dtPregao"] == dt_max)].set_index(["cdAcao"])
dfAvgVol5d  = busca_media(df5d, "vrVolume", "vol").set_index(["cdAcao"])
dfAvgVol25d = busca_media(df25d, "vrVolume", "vol").set_index(["cdAcao"])

In [ ]:
 dfPc5d = pd.DataFrame({
         "dtInicio":dfDtMin5d["dtPregao"] ,"dtFim":dfDtMax["dtPregao"]
        ,"vrInicio":dfDtMin5d["vrFech"]   ,"vrFim":dfDtMax["vrFech"]
        ,"pcPeriodo":((dfDtMax["vrFech"] - dfDtMin5d["vrFech"]) / dfDtMin5d["vrFech"]) * 100
        ,"avgVol":dfAvgVol5d["vol"]
    })

In [ ]:
 dfPc25d = pd.DataFrame({
         "dtInicio":dfDtMin25d["dtPregao"] ,"dtFim":dfDtMax["dtPregao"]
        ,"vrInicio":dfDtMin25d["vrFech"]   ,"vrFim":dfDtMax["vrFech"]
        ,"pcPeriodo":((dfDtMax["vrFech"] - dfDtMin25d["vrFech"]) / dfDtMin25d["vrFech"]) * 100
        ,"avgVol":dfAvgVol25d["vol"]
    })

In [ ]:
# dfPc25d.sort_values(["pcPeriodo"], ascending=False).loc[(dfPc25d["avgVol"] > 6000000)]

In [ ]:
# dfPc5d.sort_values(["pcPeriodo"], ascending=False).loc[(dfPc5d["avgVol"] > 6000000)]

In [ ]:
# dfPc25d['pcPeriodo']

In [ ]:
df_temp = monta_lucro_periodo(df, 25, 0, True)
# monta_lucro_periodo(df, 25, 0)

In [ ]:
# df_temp

In [ ]:
df_n_dias = busca_periodos(df, 26)

dias_ant = 2
i = 0
while i <= dias_ant:
    dt_max  = df_n_dias["dtPregao"].max()
    df_n_dias = df_n_dias.loc[df_n_dias["dtPregao"] != dt_max]
    i += 1
    
dt_max

In [ ]:
### 25 dias, Order by 0.5% # df05.loc[(df05["AvgVol"] > 6000000) & (df05["0.5%"] >= 23) & (df05["AvgVrFech"] > 5)].sort_values(["0.5%", "1.0%", "1.5%", "2.0%"], ascending=False)
### 20 dias, Order by 0.5% # df05_20d.loc[(df05_20d["AvgVol"] > 6000000) & (df05_20d["0.5%"] >= 19) & (df05_20d["AvgVrFech"] > 5)].sort_values(["0.5%", "1.0%", "1.5%", "2.0%"], ascending=False)
### 15 dias, Order by 0.5% # df05_15d.loc[(df05_15d["AvgVol"] > 6000000) & (df05_15d["0.5%"] >= 15) & (df05_15d["AvgVrFech"] > 5)].sort_values(["0.5%", "1.0%", "1.5%", "2.0%"], ascending=False)

## Definindo valores para compra

In [ ]:
# Valor disponível para cmp / 1000
# Valor da ação no leilão
vr_corretagem = 9.8
emolumentos = 0.000325
acao = 'LIGT3'
vr_disp = 110
vr_acao_leilao = 1148 / 100
vr_qtd = round((vr_disp * 1000) / (vr_acao_leilao * 100)) * 100

In [ ]:
vr_cmp = round(vr_acao_leilao * 1.014, 2)

vr_tot_cmp = vr_acao_leilao * vr_qtd
corret_cmp = vr_corretagem + (vr_tot_cmp * emolumentos)
vr_tot_liq_cmp = vr_tot_cmp + corret_cmp

vr_acao_vnd_10 = float('{0:.2f}'.format(vr_acao_leilao * 1.0108 + 0.01))
vr_tot_vnd_10 = vr_acao_vnd_10 * vr_qtd
corret_vnd_10 = vr_corretagem + (vr_tot_vnd_10 * emolumentos)
vr_tot_liq_vnd_10 = vr_tot_vnd_10 - corret_vnd_10

vr_acao_vnd_15 = float('{0:.2f}'.format(vr_acao_leilao * 1.0158 + 0.01))
vr_tot_vnd_15 = vr_acao_vnd_15 * vr_qtd
corret_vnd_15 = vr_corretagem + (vr_tot_vnd_15 * emolumentos)
vr_tot_liq_vnd_15 = vr_tot_vnd_15 - corret_vnd_15

In [ ]:
d = {'Compra':     ['Quantidade', 'Valor Leilão', 'Total', 'Total líquido', 'Corretagem Compra'],
     ' ':          [vr_qtd, vr_acao_leilao, vr_tot_cmp, vr_tot_liq_cmp, corret_cmp],
     'Venda 1,0%': ['Valor compra', 'Valor', 'Total', 'Total líquido', 'Corretagem Venda'],
     '1,0%':       [vr_cmp, vr_acao_vnd_10, vr_tot_vnd_10, vr_tot_liq_vnd_10, corret_vnd_10],
     'Venda 1,5%': [' ', 'Valor', 'Total', 'Total líquido', 'Corretagem Venda'],
     '1,5%':       [' ', vr_acao_vnd_15, vr_tot_vnd_15, vr_tot_liq_vnd_15, corret_vnd_15]
    }
df_calc = pd.DataFrame(data=d)
df_calc

In [ ]:
d2 = {'Index':      ['Quantidade', 'Valor', 'Total', 'Total líquido', 'Corretagem'],
      'Compra':     [vr_qtd, vr_cmp, vr_tot_cmp, vr_tot_liq_cmp, corret_cmp],
      'Venda 1.0%': [vr_qtd, vr_acao_vnd_10, vr_tot_vnd_10, vr_tot_liq_vnd_10, corret_vnd_10],
      'Venda 1.5%': [vr_qtd, vr_acao_vnd_15, vr_tot_vnd_15, vr_tot_liq_vnd_15, corret_vnd_15]
     }
df_calc2 = pd.DataFrame(data=d2)
df_calc2.set_index('Index')
df_calc2

In [ ]:
time.asctime( time.localtime(time.time()) )